In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime 
import seaborn as sns
%matplotlib inline

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import explained_variance_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import LassoCV
from sklearn.linear_model import RidgeCV
from sklearn.utils import shuffle
import statsmodels.api as sm

/anaconda/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [3]:
def set_data(file):
    bikeshare_machine = pd.read_csv(file, 
                        parse_dates=['Start date', 'End date'])
    bikeshare_machine.drop('Unnamed: 0', 1, inplace=True)
    included_cols = ['start_station','end_station','Member Type','time_diff','season','mnth','holiday',
                     'weekday','workingday','weathersit','temp','hum','windspeed','miles',
                     'rush_hour','metro_dist','landmark_dist_start','landmark_dist_end']
    bikeshare_machine = bikeshare_machine[included_cols]
    bikeshare_machine['season'] = bikeshare_machine['season'].astype('category')
    bikeshare_machine['mnth'] = bikeshare_machine['mnth'].astype('category')
    bikeshare_machine['holiday'] = bikeshare_machine['holiday'].astype('category')
    bikeshare_machine['weekday'] = bikeshare_machine['weekday'].astype('category')
    bikeshare_machine['workingday'] = bikeshare_machine['workingday'].astype('category')
    bikeshare_machine['weathersit'] = bikeshare_machine['weathersit'].astype('category')
    bikeshare_machine['Member Type'] = bikeshare_machine['Member Type'].astype('category')
    bikeshare_machine['start_station'] = bikeshare_machine['start_station'].astype('category')
    bikeshare_machine['end_station'] = bikeshare_machine['end_station'].astype('category')
    bikeshare_machine['rush_hour'] = bikeshare_machine['rush_hour'].astype('category')
    col_names = ['start_station', 'end_station','member_type','time_diff','season','month','holiday',
             'weekday','work_day','weather_cat','temperature','humidity','windspeed','miles','rush_hour',
                'metro_dist','landmark_dist_start','landmark_dist_end']
    bikeshare_machine.columns = col_names
    tmin = -8
    tmax = 39
    hum_max = 100
    wind_max = 67
    bikeshare_machine['temp'] = bikeshare_machine['temperature'] * (tmax - tmin) + tmin
    bikeshare_machine['hum'] = bikeshare_machine['humidity'] * 100
    bikeshare_machine['wind'] = bikeshare_machine['windspeed'] * 67
    bikeshare_machine = pd.get_dummies(bikeshare_machine, 
                                 columns=['rush_hour','member_type','holiday','work_day'], drop_first=True)
    bikeshare_machine = pd.get_dummies(bikeshare_machine, 
                                 columns=['start_station','end_station','season','month','weekday','weather_cat'])
    bikeshare_machine = shuffle(bikeshare_machine)
    return bikeshare_machine

In [4]:
file = '/Users/matthewcassi/Documents/Bike-Sharing-Dataset/Bikeshare_Time_Prediction/Casual_RushMetro/landmark_casual.csv'
bikeshare_machine = set_data(file)

In [5]:
bikeshare_machine = bikeshare_machine.drop(['temperature', 'humidity', 'windspeed'], 1)
bikeshare_machine = bikeshare_machine.rename(columns = {'member_type_Registered':'member_type'})
bikeshare_machine.head()

,time_diff,miles,metro_dist,landmark_dist_start,landmark_dist_end,temp,hum,wind,rush_hour_1,holiday_1,...,weekday_0,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weather_cat_1,weather_cat_2,weather_cat_3
137456,29.650,0.921043,0.351986,0.624320,0.322607,4.141651,56.0833,16.292189,0,0,...,0,0,0,0,0,0,1,0,1,0
154583,9.750,0.972359,0.149435,0.356938,0.604853,11.818349,74.1250,6.667338,0,0,...,1,0,0,0,0,0,0,1,0,0
9252,5.983,0.708713,0.850354,2.691478,2.004944,8.143466,68.0000,8.391616,0,0,...,1,0,0,0,0,0,0,1,0,0
5264,22.400,0.000000,0.303549,0.401587,0.401587,11.505000,60.2917,14.041793,0,0,...,0,0,0,0,1,0,0,1,0,0
127327,21.933,0.911021,0.133167,0.328533,0.356938,27.250000,65.1250,10.666400,0,0,...,0,0,0,0,1,0,0,1,0,0


### Model 1 - Remove Some Variables that are correlated

In [6]:
# Leave workday, drop weekdays, leave season, drop month
# Workday overlaps with workday/not workday and months overlap with seasons
remove_cols = ['weekday_0', 'weekday_1','weekday_2','weekday_3','weekday_4','weekday_5','weekday_6', 
              'month_1','month_2','month_3','month_4','month_5','month_6','month_7','month_8','month_9',
              'month_10','month_11','month_12','time_diff']
X1 = np.matrix(bikeshare_machine.drop(remove_cols, 1))
y1 = bikeshare_machine['time_diff']

In [7]:
# Split the data into training and testing sets and check the shape
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size = 0.25, random_state=17)
X1_train.shape, X1_test.shape, y1_train.shape, y1_test.shape

((152719, 275), (50907, 275), (152719,), (50907,))

In [8]:
# Fit model
model1 = LinearRegression()
model1.fit(X1_train, y1_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [9]:
pred1 = model1.predict(X1_test)
cvscores_model1 = cross_val_score(model1, X1_train, y1_train, cv=5)
model1_r = model1.score(X1_test, y1_test)
model1_mse = mean_squared_error(y1_test, pred1)
model1_rmse = np.sqrt(model1_mse)
adjustedr1 = 1 - (1-model1_r)*(len(y1_test)-1)/(len(y1_test)-X1_test.shape[1]-1)

In [10]:
cvscores_model1, model1_r, adjustedr1, model1_mse, model1_rmse

(array([ -1.13697684e+13,   1.58284768e-01,   1.50719352e-01,
          1.52664989e-01,   1.50413459e-01]),
 0.15306020546185239,
 0.14846009004841032,
 283.00409722626279,
 16.822725618230322)

In [11]:
model1_sm = sm.OLS(y1_train, X1_train.astype(float)).fit()
model1_sm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              time_diff   R-squared:                       0.158
Model:                            OLS   Adj. R-squared:                  0.156
Method:                 Least Squares   F-statistic:                     106.3
Date:                Sat, 16 Dec 2017   Prob (F-statistic):               0.00
Time:                        14:20:27   Log-Likelihood:            -6.4699e+05
No. Observations:              152719   AIC:                         1.295e+06
Df Residuals:                  152450   BIC:                         1.297e+06
Df Model:                         268                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             3.9929      0.055     72.178      0.000       3.885       4.101
x2            -1.1384      0.948     -1.201      0.230      -2.996       0.719
x3             1.2270      0.509      2.408      0.016       0.228       2.226
x4             0.1873      0.466      0.402      0.687      -0.725       1.100
x5             0.0560      0.009      5.944      0.000       0.038       0.075
x6            -0.0008      0.004     -0.197      0.844      -0.009       0.008
x7             0.0179      0.010      1.820      0.069      -0.001       0.037
x8            -3.9690      0.209    -18.998      0.000      -4.378      -3.559
x9            -0.8466      0.241     -3.515      0.000      -1.319      -0.375
x10           -2.8033      0.091    -30.677      0.000      -2.982      -2.624
x11            1.0035      1.551      0.647      0.518      -2.037       4.044
x12           -4.2127      0.615     -6.846      0.000      -5.419      -3.007
x13            4.7315      0.318     14.862      0.000       4.107       5.355
x14           -5.7539      1.632     -3.526      0.000      -8.952      -2.556
x15           -5.7572      0.744     -7.735      0.000      -7.216      -4.298
x16            5.4960      1.368      4.017      0.000       2.815       8.178
x17           -2.1207      1.889     -1.123      0.262      -5.823       1.582
x18           -6.0736      0.699     -8.690      0.000      -7.443      -4.704
x19           -3.7772      0.745     -5.070      0.000      -5.237      -2.317
x20            1.1955      0.447      2.674      0.007       0.319       2.072
x21            5.5756      0.399     13.982      0.000       4.794       6.357
x22           -5.4498      0.839     -6.493      0.000      -7.095      -3.805
x23            2.9226      0.420      6.956      0.000       2.099       3.746
x24           -5.4487      0.672     -8.113      0.000      -6.765      -4.132
x25           -2.6925      0.476     -5.654      0.000      -3.626      -1.759
x26           -1.0984      0.410     -2.682      0.007      -1.901      -0.296
x27           -3.8580      0.422     -9.139      0.000      -4.685      -3.031
x28           -5.1430      1.055     -4.875      0.000      -7.211      -3.075
x29           -2.2010      1.445     -1.523      0.128      -5.033       0.631
x30            4.8153      0.953      5.055      0.000       2.948       6.682
x31           -1.4562      0.375     -3.880      0.000      -2.192      -0.721
x32           -7.9771      2.002     -3.984      0.000     -11.902      -4.052
x33           -4.1582      0.577     -7.210      0.000      -5.289      -3.028
x34           -2.2242      0.411     -5.418      0.000      -3.029      -1.420
x35            1.6271      0.529      3.075      0.002       0.590       2.664
x36           -1.8666      0.434     -4.305      0.000      -2.716      -1.017
x3

### Model 2 - Try reverse of Model 1

In [12]:
# Drop workday, leave weekdays, drop season, leave month
# Workday overlaps with workday/not workday and months overlap with seasons
remove_cols = ['work_day_1','season_1', 'season_2', 'season_3', 'season_4','time_diff']
X2 = np.matrix(bikeshare_machine.drop(remove_cols, 1))
y2 = bikeshare_machine['time_diff']

In [13]:
# Split the data into training and testing sets and check the shape
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size = 0.25, random_state=17)
X2_train.shape, X2_test.shape, y2_train.shape, y2_test.shape

((152719, 289), (50907, 289), (152719,), (50907,))

In [14]:
# Fit model
model2 = LinearRegression()
model2.fit(X2_train, y2_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [15]:
pred2 = model2.predict(X2_test)
cvscores_model2 = cross_val_score(model2, X2_train, y2_train, cv=5)
model2_r = model2.score(X2_test, y2_test)
model2_mse = mean_squared_error(y2_test, pred2)
model2_rmse = np.sqrt(model2_mse)
adjustedr2 = 1 - (1-model2_r)*(len(y2_test)-1)/(len(y2_test)-X2_test.shape[1]-1)

In [16]:
cvscores_model2, model2_r, adjustedr2, model2_mse, model2_rmse

(array([ -2.47918979e+09,   1.60891196e-01,   1.51986701e-01,
          1.55052123e-01,   1.52189244e-01]),
 0.15588225145678969,
 0.15106272384098896,
 282.06111333971182,
 16.79467514838295)

In [17]:
# Use statsmodel to check the statistical side of the model
model2_sm = sm.OLS(y2_train, X2_train.astype(float)).fit()
model2_sm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              time_diff   R-squared:                       0.160
Model:                            OLS   Adj. R-squared:                  0.158
Method:                 Least Squares   F-statistic:                     103.1
Date:                Sat, 16 Dec 2017   Prob (F-statistic):               0.00
Time:                        14:20:50   Log-Likelihood:            -6.4679e+05
No. Observations:              152719   AIC:                         1.294e+06
Df Residuals:                  152437   BIC:                         1.297e+06
Df Model:                         281                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             3.9894      0.055     72.179      0.000       3.881       4.098
x2            -1.2002      0.947     -1.268      0.205      -3.055       0.655
x3             1.2175      0.509      2.393      0.017       0.220       2.215
x4             0.2107      0.465      0.453      0.651      -0.701       1.122
x5             0.1150      0.014      8.307      0.000       0.088       0.142
x6            -0.0142      0.005     -2.955      0.003      -0.024      -0.005
x7            -0.0203      0.010     -1.995      0.046      -0.040      -0.000
x8            -3.8671      0.209    -18.518      0.000      -4.276      -3.458
x9             1.5270      0.265      5.753      0.000       1.007       2.047
x10            0.9105      1.550      0.588      0.557      -2.127       3.948
x11           -4.2097      0.615     -6.849      0.000      -5.414      -3.005
x12            4.6470      0.318     14.618      0.000       4.024       5.270
x13           -5.8357      1.630     -3.581      0.000      -9.030      -2.641
x14           -5.6881      0.744     -7.650      0.000      -7.145      -4.231
x15            5.4816      1.367      4.011      0.000       2.803       8.160
x16           -2.1135      1.887     -1.120      0.263      -5.812       1.585
x17           -6.0052      0.698     -8.601      0.000      -7.374      -4.637
x18           -3.7350      0.744     -5.019      0.000      -5.194      -2.276
x19            1.1677      0.447      2.615      0.009       0.293       2.043
x20            5.5504      0.398     13.938      0.000       4.770       6.331
x21           -5.4415      0.838     -6.491      0.000      -7.085      -3.798
x22            2.8779      0.420      6.859      0.000       2.056       3.700
x23           -5.4149      0.671     -8.072      0.000      -6.730      -4.100
x24           -2.6891      0.476     -5.653      0.000      -3.621      -1.757
x25           -1.0756      0.409     -2.628      0.009      -1.878      -0.273
x26           -3.8093      0.422     -9.034      0.000      -4.636      -2.983
x27           -5.0800      1.054     -4.821      0.000      -7.145      -3.015
x28           -1.9817      1.443     -1.373      0.170      -4.810       0.846
x29            4.7977      0.952      5.042      0.000       2.933       6.663
x30           -1.4317      0.375     -3.817      0.000      -2.167      -0.696
x31           -7.9825      2.000     -3.991      0.000     -11.903      -4.062
x32           -4.1067      0.576     -7.129      0.000      -5.236      -2.978
x33           -2.1660      0.410     -5.279      0.000      -2.970      -1.362
x34            1.6243      0.528      3.074      0.002       0.589       2.660
x35           -1.8640      0.433     -4.305      0.000      -2.713      -1.015
x36           -0.4865      1.035     -0.470      0.638      -2.516       1.543
x3

### Model 3 - All variables

In [18]:
# Keep all but time_diff
X3 = np.matrix(bikeshare_machine.drop('time_diff', 1))
y3 = bikeshare_machine['time_diff']

In [19]:
# Split the data into training and testing sets and check the shape
X3_train, X3_test, y3_train, y3_test = train_test_split(X3, y3, test_size = 0.25, random_state=17)
X3_train.shape, X3_test.shape, y3_train.shape, y3_test.shape

((152719, 294), (50907, 294), (152719,), (50907,))

In [20]:
# Fit model
model3 = LinearRegression()
model3.fit(X3_train, y3_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [21]:
pred3 = model3.predict(X3_test)
cvscores_model3 = cross_val_score(model3, X3_train, y3_train, cv=5)
model3_r = model3.score(X3_test, y3_test)
model3_mse = mean_squared_error(y3_test, pred3)
model3_rmse = np.sqrt(model3_mse)
adjustedr3 = 1 - (1-model3_r)*(len(y3_test)-1)/(len(y3_test)-X3_test.shape[1]-1)

In [22]:
cvscores_model3, model3_r, adjustedr3, model3_mse, model3_rmse

(array([ -1.86163082e+12,   1.60870816e-01,   1.51922779e-01,
          1.55234004e-01,   1.52250207e-01]),
 0.15591030472442347,
 0.15100707287405168,
 282.05173936799372,
 16.794396070356139)

In [23]:
# Use statsmodel to check the statistical side of the model
model3_sm = sm.OLS(y3_train, X3_train.astype(float)).fit()
model3_sm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              time_diff   R-squared:                       0.160
Model:                            OLS   Adj. R-squared:                  0.158
Method:                 Least Squares   F-statistic:                     102.0
Date:                Sat, 16 Dec 2017   Prob (F-statistic):               0.00
Time:                        14:21:18   Log-Likelihood:            -6.4678e+05
No. Observations:              152719   AIC:                         1.294e+06
Df Residuals:                  152434   BIC:                         1.297e+06
Df Model:                         284                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             3.9885      0.055     72.164      0.000       3.880       4.097
x2            -1.4098      0.949     -1.485      0.138      -3.271       0.451
x3             1.0872      0.510      2.134      0.033       0.089       2.086
x4             0.0843      0.466      0.181      0.857      -0.830       0.998
x5             0.1134      0.014      8.178      0.000       0.086       0.141
x6            -0.0135      0.005     -2.810      0.005      -0.023      -0.004
x7            -0.0220      0.010     -2.153      0.031      -0.042      -0.002
x8            -3.8605      0.209    -18.486      0.000      -4.270      -3.451
x9             3.7230      0.229     16.259      0.000       3.274       4.172
x10            2.2468      0.157     14.355      0.000       1.940       2.554
x11            0.9458      1.550      0.610      0.542      -2.091       3.983
x12           -4.3524      0.615     -7.076      0.000      -5.558      -3.147
x13            4.4197      0.320     13.798      0.000       3.792       5.048
x14           -5.8761      1.630     -3.606      0.000      -9.070      -2.682
x15           -5.7202      0.743     -7.695      0.000      -7.177      -4.263
x16            5.4507      1.366      3.989      0.000       2.773       8.129
x17           -2.0710      1.887     -1.097      0.272      -5.769       1.628
x18           -6.0223      0.698     -8.626      0.000      -7.391      -4.654
x19           -3.7063      0.745     -4.978      0.000      -5.166      -2.247
x20            0.9544      0.448      2.131      0.033       0.076       1.833
x21            5.3387      0.400     13.354      0.000       4.555       6.122
x22           -5.4987      0.838     -6.560      0.000      -7.142      -3.856
x23            2.6473      0.421      6.281      0.000       1.821       3.473
x24           -5.5103      0.671     -8.214      0.000      -6.825      -4.195
x25           -2.8283      0.476     -5.946      0.000      -3.761      -1.896
x26           -1.1770      0.409     -2.879      0.004      -1.978      -0.376
x27           -3.9539      0.422     -9.367      0.000      -4.781      -3.127
x28           -5.0187      1.054     -4.761      0.000      -7.085      -2.953
x29           -1.7207      1.445     -1.191      0.234      -4.552       1.111
x30            4.7496      0.951      4.992      0.000       2.885       6.614
x31           -1.5511      0.375     -4.141      0.000      -2.285      -0.817
x32           -8.0491      2.000     -4.024      0.000     -11.969      -4.129
x33           -4.1863      0.576     -7.269      0.000      -5.315      -3.058
x34           -2.3436      0.411     -5.706      0.000      -3.149      -1.539
x35            1.3913      0.530      2.624      0.009       0.352       2.431
x36           -2.0747      0.435     -4.774      0.000      -2.926      -1.223
x3

### Model 4 - Ridge with Round 1

In [24]:
# Fit the model with 5 folds
alpha = np.arange(0.0001, 20, 25)
param_grid = {'alpha': alpha}
ridge1 = Ridge(fit_intercept=True)
ridge1_gs = GridSearchCV(ridge1, param_grid, cv=5)
ridge1_gs.fit(X1_train, y1_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'alpha': array([ 0.0001])}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score=True, scoring=None, verbose=0)

In [25]:
pred4 = ridge1_gs.predict(X1_test)
ridge1_r = ridge1_gs.score(X1_test, y1_test)
ridge1_mse = mean_squared_error(y1_test, pred4)
ridge1_rmse = np.sqrt(ridge1_mse)
adjustedr4 = 1 - (1-ridge1_r)*(len(y1_test)-1)/(len(y1_test)-X1_test.shape[1]-1)

In [26]:
ridge1_r, ridge1_mse, ridge1_rmse, adjustedr4

(0.15306123299937591,
 283.00375387561246,
 16.822715413262284,
 0.1484611231669577)

### Ridge with Round 2

In [27]:
# Fit the model with 5 folds
alpha = np.arange(0.0001, 20, 25)
param_grid = {'alpha': alpha}
ridge2 = Ridge(fit_intercept=True)
ridge2_gs = GridSearchCV(ridge2, param_grid, cv=5)
ridge2_gs.fit(X2_train, y2_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'alpha': array([ 0.0001])}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score=True, scoring=None, verbose=0)

In [28]:
pred5 = ridge2_gs.predict(X2_test)
ridge2_r = ridge2_gs.score(X2_test, y2_test)
ridge2_mse = mean_squared_error(y2_test, pred5)
ridge2_rmse = np.sqrt(ridge2_mse)
adjustedr5 = 1 - (1-ridge2_r)*(len(y2_test)-1)/(len(y2_test)-X2_test.shape[1]-1)

In [29]:
ridge2_r, ridge2_mse, ridge2_rmse, adjustedr5

(0.15588155077892696, 282.0613474705288, 16.794682118769881, 0.151062019162575)

### Ridge with all data

In [30]:
# Fit the model with 5 folds
alpha = np.arange(0.0001, 20, 25)
param_grid = {'alpha': alpha}
ridge3 = Ridge(fit_intercept=True)
ridge3_gs = GridSearchCV(ridge3, param_grid, cv=5)
ridge3_gs.fit(X3_train, y3_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'alpha': array([ 0.0001])}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score=True, scoring=None, verbose=0)

In [31]:
pred6 = ridge3_gs.predict(X3_test)
ridge3_r = ridge3_gs.score(X3_test, y3_test)
ridge3_mse = mean_squared_error(y3_test, pred6)
ridge3_rmse = np.sqrt(ridge3_mse)
adjustedr6 = 1 - (1-ridge3_r)*(len(y3_test)-1)/(len(y3_test)-X3_test.shape[1]-1)

In [32]:
ridge3_r, ridge3_mse, ridge3_rmse, adjustedr6

(0.15591028689087227,
 282.05174532705729,
 16.7943962477684,
 0.15100705493690714)